###### Solar Flare GP Model with Dynesty Sampling

I want to see whether I can sample the parameter space using Dynesty.

Let's try!

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
sns.set_context('talk')

import numpy as np
import pandas as pd

import celerite

/home/chris/.local/lib/python2.7/site-packages/scipy/special/__init__.py:640: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ufuncs import *
/home/chris/.local/lib/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/home/chris/.local/lib/python2.7/site-packages/scipy/linalg/__init__.py:207: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *
/home/chris/.local/lib/python2.7/site-packages/scipy/special/_ellip_harm.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ellip_harm_2 import _ellipsoid, _ellipsoid_norm
/home/chris/.local/lib/python2.7/site-packages/scipy/interpolate/_bsplines.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary

Let's simulate a data set, using the code from the QPPStrengthTest notebook:

In [2]:
log_s0 = np.log(1e5) # QPP amplitude
log_q0 = np.log(10.0) # QPP quality factor
log_omega0 = 5

I'm going to start simulating a GP without the OU process:

In [3]:
# these should all be within priors!
#realparams = [-10, -10.0] # OU process
qpoparams = [log_s0, log_q0, log_omega0] # QPO
modelparams = [9.0, 6.92311406, 6.85207764, np.log(1000)] # flare model
#mean_val = 1000.0
trueparams = np.hstack([qpoparams, modelparams])#+ modelparams # + realparams # combined set of parameters

In [4]:
import QPP_Funcs as qpp

/home/chris/.local/lib/python2.7/site-packages/scipy/ndimage/measurements.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _ni_label
/home/chris/.local/lib/python2.7/site-packages/h5py/__init__.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/home/chris/.local/lib/python2.7/site-packages/h5py/__init__.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import h5a, h5d, h5ds, h5f, h5fd, h5g, h5r, h5s, h5t, h5p, h5z
/home/chris/.local/lib/python2.7/site-packages/h5py/_hl/group.py:22: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .. import h5g, h5i, h5o, h5r, h5t, h5l, h5p
/home/chris/.local/lib/python2.7/site-packages/dynesty/bounding.py:73: UserWarning: Robust OAS shrunk co

In [5]:
from celerite.modeling import Model

class CTSModel_prior(Model):
    name="CTSModel_prior"
    parameter_names = ("log_A", "log_tau1", "log_tau2", "log_bkg")
 
    def get_value(self, t):
        A = np.exp(self.log_A)
        tau1 = np.exp(self.log_tau1)
        tau2 = np.exp(self.log_tau2)
        lam = np.exp(np.sqrt(2*np.exp(tau1/tau2)))
        bkg = np.exp(self.log_bkg)
        return A*lam*np.exp((-tau1/t)-(t/tau2)) + bkg
    
    #the gradient terms were manually calculated
    def compute_gradient(self, t):
        A = np.exp(self.log_A)
        tau1 = np.exp(self.log_tau1)
        tau2 = np.exp(self.log_tau2)
        lam = np.exp(np.sqrt(2*np.exp(tau1/tau2)))
        dA = (1./A) * self.get_value(t)
        dtau1 = ((1/(np.sqrt(2*tau1*tau2))) - (1/t)) * self.get_value(t)
        dtau2 = ((t/(tau2**2))-(tau1/((tau2**2)*np.sqrt(2*tau1/tau2)))) * self.get_value(t)
        return np.array([dA, dtau1, dtau2])
        

    #defining our somewhat naive prior, a simple tophat distribution for each parameter
    #SUBJECT TO CHANGE!!!

    def log_prior(self):
        probA = 1.
        probtau1 = 1.
        probtau2 = 1.
        if not (self.log_A>1 and self.log_A<25): 
            probA = 0.
        if not ((self.log_tau1>1 and self.log_tau1<15)):
            probtau1 = 0.
        if not ((self.log_tau2>1 and self.log_tau2<15)):
            probtau2 = 0.
        return np.log(probA * probtau1 * probtau2 * np.e)



Simulating the data ...

In [6]:
qpolabel = "A{:1.2f}".format(qpoparams[0])+"Q{:1.2f}".format(qpoparams[1])+"w{:1.2f}".format(qpoparams[2])

model = CTSModel_prior(log_A = modelparams[0], log_tau1 = modelparams[1], 
                       log_tau2 = modelparams[2], log_bkg = modelparams[3])
kernel1 = qpp.SHOTerm_Prior(log_S0 = qpoparams[0], log_Q = qpoparams[1], log_omega0 = qpoparams[2])
#kernel2 = qpp.RealTerm_Prior(log_a = realparams[0], log_c = realparams[1])
kernel = kernel1 #kernel2 + kernel1

dt = 4.0
tseg = 4000.0

time = np.linspace(0,tseg,tseg/dt)+0.01
#counts = qpp.simulate(time, model, kernel, noisy = True)

K = kernel.get_value(time[:, None] - time[None, :])
#y = np.abs(np.random.multivariate_normal(model.get_value(x), K))
y = np.abs(np.random.multivariate_normal(model.get_value(time), K))
yerr = 10
counts = np.random.normal(y, yerr)

/home/chris/.local/lib/python2.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
  if sys.path[0] == '':
/home/chris/.local/lib/python2.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: covariance is not symmetric positive-semidefinite.


In [7]:

counts_err = np.sqrt(counts)

In [8]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9,4))
ax1.errorbar(time, counts, yerr=counts_err, fmt="o", markersize=8, lw=1, color="black")
ax1.plot(time, counts, lw=1, color="black", alpha=0.5)
ax1.set_xlim(time[0], time[-1])
ax1.set_ylim(0, np.max(counts)*1.1)
ax1.set_title("Full light curve")

ax2.errorbar(time, counts, yerr=counts_err, fmt="o", markersize=8, lw=1, color="black")
ax2.set_xlim(900, 1100)
ax2.set_title("short segment around peak")
ax2.set_ylim(np.max(counts)*0.9, np.max(counts)*1.1)

plt.tight_layout()

<IPython.core.display.Javascript object>

This QPO is super strong, just so that we know what's going on (and that it's really visible; you can see it with your naked eye if you zoom in!). 

Now we need to set up the model:

In [9]:
A_guess, t1_guess, t2_guess = qpp.initguess(time,counts)
model = CTSModel_prior(log_A = np.log(A_guess), log_tau1 = np.log(t1_guess), 
                           log_tau2 = np.log(t2_guess), log_bkg=np.log(1000))
kernel1 = qpp.SHOTerm_Prior(log_S0 =np.log(1e3), log_Q = np.log(10), log_omega0 = 3) #write guesser for kernel parameters
#kernel2 = qpp.RealTerm_Prior(log_a=0., log_c=0.) #write guesser for kernel parameters
kernel = kernel1
gp = celerite.GP(kernel, mean=model, fit_mean=True)
gp.compute(time, np.sqrt(counts))
initparams = gp.get_parameter_vector()

Let's run an optimizer:

In [10]:
soln = qpp.optimize_gp(gp, counts)
gp.set_parameter_vector(soln.x)
figopt = qpp.plot_gp(time, counts, np.sqrt(counts), gp, model, predict=True, label = "Optimized fit")

QPP_Funcs.py:90: RuntimeWarning: divide by zero encountered in log
  return np.log(prob_S0*prob_Q*prob_omega0 * np.e)


<IPython.core.display.Javascript object>

In [11]:
trueparams

array([11.51292546,  2.30258509,  5.        ,  9.        ,  6.92311406,
        6.85207764,  6.90775528])

In [12]:
gp.set_parameter_vector(trueparams)

In [13]:
figtrue = qpp.plot_gp(time, counts, np.sqrt(counts), gp, model, predict=True, label="True Parameters")

<IPython.core.display.Javascript object>

In [14]:
soln.x

array([7.51715217, 2.91375448, 4.1366466 , 9.7262616 , 6.39704683,
       6.59521574, 6.90874722])

In [15]:
#sampling
sampler = qpp.sample_gp(soln.x, counts, gp, burnin=3000, nwalkers=100, nsteps = 3000)
chain = sampler.chain


figsam = qpp.plot_chain(chain, labels = gp.get_parameter_names(), burstid = qpolabel)

/home/chris/.local/lib/python2.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in log
/home/chris/.local/lib/python2.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in exp
  # This is added back by InteractiveShellApp.init_path()
/home/chris/.local/lib/python2.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in multiply
  del sys.path[0]
/home/chris/.local/lib/python2.7/site-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in double_scalars
  del sys.path[0]


emcee: Exception while calling your likelihood function:
  params: [13.02329143  1.59487712  3.43873613  7.92869446  4.49839428  9.01114431
  7.97018996]
  args: (array([1.61524493e+04, 3.66745404e+03, 8.86732119e+02, 2.90654968e+03,
       1.24660818e+04, 4.41852272e+03, 1.01645179e+03, 1.56921926e+04,
       1.68926761e+04, 4.15823562e+03, 6.46352468e+03, 1.44848244e+04,
       9.66471104e+02, 8.13177930e+03, 1.49198093e+04, 1.08197320e+04,
       8.34192871e+03, 6.23174036e+03, 8.73280275e+03, 8.79121783e+03,
       1.18658536e+04, 1.93446080e+03, 1.43150559e+04, 7.76057765e+02,
       2.21511382e+04, 2.94058116e+03, 9.78627785e+03, 6.33126754e+02,
       1.86507231e+04, 1.32965161e+04, 2.16544734e+04, 6.04495420e+02,
       1.80543402e+03, 2.27774955e+03, 2.19011196e+03, 7.54029542e+03,
       5.84746904e+03, 4.10338593e+03, 8.92956650e+03, 1.86386764e+03,
       6.32776494e+03, 1.59766058e+04, 2.24734590e+04, 1.53806266e+03,
       2.44271668e+03, 2.16097762e+04, 9.99456541e+03, 2

Traceback (most recent call last):
  File "/home/chris/.local/lib/python2.7/site-packages/emcee/ensemble.py", line 519, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "QPP_Funcs.py", line 161, in log_probability
    gp.set_parameter_vector(params)
  File "/home/chris/.local/lib/python2.7/site-packages/celerite/modeling.py", line 239, in set_parameter_vector
    self.parameter_vector = v
  File "/home/chris/.local/lib/python2.7/site-packages/celerite/modeling.py", line 378, in parameter_vector
    m.parameter_vector = v[i:i+l]
  File "/home/chris/.local/lib/python2.7/site-packages/celerite/modeling.py", line 160, in parameter_vector
    for k, val in zip(self.parameter_names, v):
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
import corner

In [ ]:
ndim=len(gp.get_parameter_vector())

In [ ]:
chain.shape

In [ ]:
flatchain = chain.reshape(300000, ndim)

In [ ]:
fig, ax = plt.subplots(ndim, ndim, figsize=(10, 10))
corner.corner(flatchain, truths=trueparams, labels=gp.get_parameter_names(), fig=fig);

# Dynesty Sampling

I want to implement Dynamical Nested Sampling with [Dynesty](https://dynesty.readthedocs.io/en/latest/crashcourse.html). Let's do that! 

The likelihood is just the GP log-likelihood:

In [ ]:
gp.parameter_names

In [ ]:
def loglikelihood(params):
    gp.set_parameter_vector(params)
    return gp.log_likelihood(counts)


In [ ]:
def from_prior(u):
    """
    Transform random samples from an n-dimensional uniform 
    distribution to the prior distribution
    """
    dt = 4.00 # time resolution of the light curve
    T = 4000.0 # total duration of the light curve

    #log_a = 20 * u[0] - 10.0
    #log_c = (np.log(T) + np.log(dt)) * u[1] + np.log(dt)
    
    log_S0 = (np.log(1e7) + np.log(1))*u[0] + np.log(1.0)
    log_Q = (np.log(20) + np.log(2.0)) * u[1] + np.log(2.0)
    log_omega0 = (7-2)* u[2] - 7
    
    log_A = (np.log(1e7) + np.log(1))*u[3] - np.log(1.0)
    log_tau1 = (np.log(1e4) - np.log(1))*u[4] - np.log(1.0)
    log_tau2 = (np.log(T) - np.log(1/100. * T))*u[5] + np.log(1./100 * T)
    log_bkg = 20*u[6]-10.0
    
#   return np.array([log_a, log_c, log_S0, log_Q, log_omega0, log_A, log_tau1, log_tau2])
    return np.array([log_S0, log_Q, log_omega0, log_A, log_tau1, log_tau2, log_bkg])
    

In [ ]:
def sample_from_prior(n=10000, ndim=4):
    u_all = np.random.uniform(size=(n, ndim))
    samples = []
    for u in u_all:
        samples.append(from_prior(u))
        
    return np.array(samples)

In [ ]:
prior_samples = sample_from_prior(n=10000, ndim=ndim)

In [ ]:
import corner

In [ ]:

fig, axes = plt.subplots(ndim, ndim, figsize=(10,10))
corner.corner(prior_samples, labels=gp.get_parameter_names(), truths=trueparams, fig=fig, label_kwargs={"fontsize":8});

In [ ]:
import dynesty

In [ ]:
gp.get_parameter_names()

In [ ]:
gp.set_parameter_vector(trueparams)

In [ ]:
ndim = len(gp.parameter_names)

In [ ]:
loglikelihood(trueparams)

In [ ]:
gp.log_likelihood(counts)

In [ ]:

sampler = dynesty.DynamicNestedSampler(loglikelihood, from_prior, ndim, bound="multi", sample="rwalk", nlive=1000)

In [ ]:
sampler.run_nested()

In [ ]:
res = sampler.results

In [ ]:
from dynesty import plotting as dyplot

In [ ]:
lnz_truth = ndim * -np.log(2 * 10.)  # analytic evidence solution
fig, axes = dyplot.runplot(res);  # summary (run) plot


In [ ]:
fig, axes = dyplot.traceplot(res, truths=trueparams,
                             truth_color='black', show_titles=True,
                             trace_cmap='viridis', connect=True,
                             connect_highlight=range(5))

In [ ]:
trueparams

In [ ]:
from dynesty import utils as dyfunc

In [ ]:
samples, weights = res.samples, np.exp(res.logwt - res.logz[-1])
mean, cov = dyfunc.mean_and_cov(samples, weights)


In [ ]:
new_samples = dyfunc.resample_equal(samples, weights)

In [ ]:
corner.corner(new_samples, truths=trueparams, labels=gp.get_parameter_names());

In [ ]:

# plot initial run (res1; left)
fg, ax = dyplot.cornerpoints(res, cmap='plasma', truths=trueparams)


Let's plot some examples:

In [ ]:
res.samples.shape

In [ ]:
samples, weights = res.samples, np.exp(res.logwt - res.logz[-1])
mean, cov = dyfunc.mean_and_cov(samples, weights)

new_samples = dyfunc.resample_equal(samples, weights)

In [ ]:
s = new_samples[0]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,5))

ax.errorbar(time, counts, yerr=counts_err, fmt="o", markersize=3, linewidth=1, c="black")
t = np.linspace(time[0], time[-1], 1000)


for s in new_samples[np.random.randint(len(new_samples), size=100)]:
    gp.set_parameter_vector(s)
    mu, var = gp.predict(counts, t, return_var=True)
    ax.plot(t, mu, lw=2, color=sns.color_palette()[1], alpha=0.2)
    
plt.tight_layout()

### OU Process Model

Can I model the same data with an OU process instead?

In [ ]:
A_guess, t1_guess, t2_guess = qpp.initguess(time,counts)
model = CTSModel_prior(log_A = np.log(A_guess), log_tau1 = np.log(t1_guess), 
                           log_tau2 = np.log(t2_guess), log_bkg=np.log(1000))
#kernel1 = qpp.SHOTerm_Prior(log_S0 =0.1, log_Q = 0.1, log_omega0 = 0.1) #write guesser for kernel parameters
kernel2 = qpp.RealTerm_Prior(log_a=0., log_c=0.) #write guesser for kernel parameters
kernel = kernel2
gp = celerite.GP(kernel, mean=model, fit_mean=True)
gp.compute(time, np.sqrt(counts))
initparams = gp.get_parameter_vector()

In [ ]:
def loglikelihood(params):
    gp.set_parameter_vector(params)
    return gp.log_likelihood(counts)



def from_prior_m2(u):
    """
    Transform random samples from an n-dimensional uniform 
    distribution to the prior distribution
    """
    dt = 4.00 # time resolution of the light curve
    T = 4000.0 # total duration of the light curve

    log_a = 20 * u[0] - 10.0
    log_c = (np.log(T) + np.log(dt)) * u[1] + np.log(dt)
    
    #log_S0 = (np.log(1e7) + np.log(1))*u[0] + np.log(1.0)
    #log_Q = (np.log(20) + np.log(2.0)) * u[1] + np.log(2.0)
    #log_omega0 = (7-2)* u[2] - 7
    
    log_A = (np.log(1e7) + np.log(1))*u[2] - np.log(1.0)
    log_tau1 = (np.log(1e4) - np.log(1))*u[3] - np.log(1.0)
    log_tau2 = (np.log(T) - np.log(1/100. * T))*u[4] + np.log(1./100 * T)
    log_bkg = 20*u[4]-10.0
    
#    return np.array([log_a, log_c, log_S0, log_Q, log_omega0, log_A, log_tau1, log_tau2])
    return np.array([log_a, log_c, log_A, log_tau1, log_tau2, log_bkg])
    

Now we can do the sampling on the model:

In [ ]:
ndim = len(gp.parameter_names)
print("number of dimensions: " + str(ndim))
sampler = dynesty.DynamicNestedSampler(loglikelihood, from_prior_m2, ndim, bound="multi", sample="rwalk", nlive=1000)

In [ ]:
sampler.run_nested()

In [ ]:
res2 = sampler.results

In [ ]:
lnz_truth = ndim * -np.log(2 * 10.)  # analytic evidence solution
fig, axes = dyplot.runplot(res2);  # summary (run) plot

In [ ]:

fig, axes = dyplot.traceplot(res2, truths=trueparams,
                             truth_color='black', show_titles=True,
                             trace_cmap='viridis', connect=True,
                             connect_highlight=range(5))

In [ ]:

samples, weights = res2.samples, np.exp(res2.logwt - res2.logz[-1])
mean, cov = dyfunc.mean_and_cov(samples, weights)

new_samples = dyfunc.resample_equal(samples, weights)

In [ ]:

corner.corner(new_samples, truths=trueparams, labels=gp.get_parameter_names());

In [ ]:

fig, ax = plt.subplots(1, 1, figsize=(10,5))

ax.errorbar(time, counts, yerr=counts_err, fmt="o", markersize=3, linewidth=1, c="black")
t = np.linspace(time[0], time[-1], 1000)


for s in new_samples[np.random.randint(len(new_samples), size=100)]:
    gp.set_parameter_vector(s)
    mu, var = gp.predict(counts, t, return_var=True)
    ax.plot(t, mu, lw=2, color=sns.color_palette()[1], alpha=0.2)
    
plt.tight_layout()

In [ ]:
bayes_fac1 = -5555.655

In [ ]:
bayes_fac2 = -8086.021

In [ ]:
bayes_fac1 - bayes_fac2

That's a pretty resounding success for Model 1 (with the QPO.

## A weaker QPO

Let's model a weaker QPO to see what happens:

In [ ]:

log_s0 = 8.0 # QPP amplitude
log_q0 = np.log(10.0) # QPP quality factor
log_omega0 = -3.0

# these should all be within priors!
#realparams = [-10, -10.0] # OU process
qpoparams = [log_s0, log_q0, log_omega0] # QPO
modelparams = [9.0, 6.92311406, 6.85207764, np.log(1000)] # flare model
#mean_val = 1000.0
trueparams = np.hstack([qpoparams, modelparams])#+ modelparams # + realparams # combined set of parameters
qpolabel = "A{:1.2f}".format(qpoparams[0])+"Q{:1.2f}".format(qpoparams[1])+"w{:1.2f}".format(qpoparams[2])

model = CTSModel_prior(log_A = modelparams[0], log_tau1 = modelparams[1], 
                       log_tau2 = modelparams[2], log_bkg = modelparams[3])
kernel1 = qpp.SHOTerm_Prior(log_S0 = qpoparams[0], log_Q = qpoparams[1], log_omega0 = qpoparams[2])
#kernel2 = qpp.RealTerm_Prior(log_a = realparams[0], log_c = realparams[1])
kernel = kernel1 #kernel2 + kernel1

dt = 4.0
tseg = 4000.0

time = np.linspace(0,tseg,tseg/dt)+0.01
#counts = qpp.simulate(time, model, kernel, noisy = True)

K = kernel.get_value(time[:, None] - time[None, :])
#y = np.abs(np.random.multivariate_normal(model.get_value(x), K))
y = np.abs(np.random.multivariate_normal(model.get_value(time), K))
yerr = 10
counts = np.random.normal(y, yerr)

counts_err = np.sqrt(counts)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9,4))
ax1.errorbar(time, counts, yerr=counts_err, fmt="o", markersize=8, lw=1, color="black")
ax1.plot(time, counts, lw=1, color="black", alpha=0.5)
ax1.set_xlim(time[0], time[-1])
ax1.set_ylim(0, np.max(counts)*1.1)
ax1.set_title("Full light curve")

ax2.errorbar(time, counts, yerr=counts_err, fmt="o", markersize=8, lw=1, color="black")
ax2.set_xlim(900, 1100)
ax2.set_title("short segment around peak")
ax2.set_ylim(np.max(counts)*0.9, np.max(counts)*1.1)

plt.tight_layout()

In [ ]:

A_guess, t1_guess, t2_guess = qpp.initguess(time,counts)
model = CTSModel_prior(log_A = np.log(A_guess), log_tau1 = np.log(t1_guess), 
                           log_tau2 = np.log(t2_guess), log_bkg=np.log(1000))
kernel1 = qpp.SHOTerm_Prior(log_S0 =0.1, log_Q = 0.1, log_omega0 = 0.1) #write guesser for kernel parameters
#kernel2 = qpp.RealTerm_Prior(log_a=0., log_c=0.) #write guesser for kernel parameters
kernel = kernel1
gp = celerite.GP(kernel, mean=model, fit_mean=True)
gp.compute(time, np.sqrt(counts))
initparams = gp.get_parameter_vector()


In [ ]:
def from_prior(u):
    """
    Transform random samples from an n-dimensional uniform 
    distribution to the prior distribution
    """
    dt = 4.00 # time resolution of the light curve
    T = 4000.0 # total duration of the light curve

    #log_a = 20 * u[0] - 10.0
    #log_c = (np.log(T) + np.log(dt)) * u[1] + np.log(dt)
    
    log_S0 = (np.log(1e7) + np.log(1))*u[0] + np.log(1.0)
    log_Q = (np.log(20) + np.log(2.0)) * u[1] + np.log(2.0)
    log_omega0 = (7-2)* u[2] - 7
    
    log_A = (np.log(1e7) + np.log(1))*u[3] - np.log(1.0)
    log_tau1 = (np.log(1e4) - np.log(1))*u[4] - np.log(1.0)
    log_tau2 = (np.log(T) - np.log(1/100. * T))*u[5] + np.log(1./100 * T)
    log_bkg = 20*u[6]-10.0
    
#    return np.array([log_a, log_c, log_S0, log_Q, log_omega0, log_A, log_tau1, log_tau2])
    return np.array([log_S0, log_Q, log_omega0, log_A, log_tau1, log_tau2, log_bkg])
    

def loglikelihood(params):
    gp.set_parameter_vector(params)
    return gp.log_likelihood(counts)



In [ ]:
ndim = len(gp.get_parameter_vector())
sampler = dynesty.DynamicNestedSampler(loglikelihood, from_prior, ndim, bound="multi", sample="rwalk", nlive=1000)
sampler.run_nested()


In [ ]:
res = sampler.results

lnz_truth = ndim * -np.log(2 * 10.)  # analytic evidence solution
fig, axes = dyplot.runplot(res);  # summary (run) plot



fig, axes = dyplot.traceplot(res, truths=trueparams,
                             truth_color='black', show_titles=True,
                             trace_cmap='viridis', connect=True,
                             connect_highlight=range(5))

samples, weights = res.samples, np.exp(res.logwt - res.logz[-1])
mean, cov = dyfunc.mean_and_cov(samples, weights)


new_samples = dyfunc.resample_equal(samples, weights)
corner.corner(new_samples, truths=trueparams, labels=gp.get_parameter_names());

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,5))

ax.errorbar(time, counts, yerr=counts_err, fmt="o", markersize=3, linewidth=1, c="black")
t = np.linspace(time[0], time[-1], 1000)


for s in new_samples[np.random.randint(len(new_samples), size=100)]:
    gp.set_parameter_vector(s)
    mu, var = gp.predict(counts, t, return_var=True)
    ax.plot(t, mu, lw=2, color=sns.color_palette()[1], alpha=0.2)
    
plt.tight_layout()

### The OU Process Model

In [ ]:
A_guess, t1_guess, t2_guess = qpp.initguess(time,counts)
model = CTSModel_prior(log_A = np.log(A_guess), log_tau1 = np.log(t1_guess), 
                           log_tau2 = np.log(t2_guess), log_bkg=np.log(1000))
#kernel1 = qpp.SHOTerm_Prior(log_S0 =0.1, log_Q = 0.1, log_omega0 = 0.1) #write guesser for kernel parameters
kernel2 = qpp.RealTerm_Prior(log_a=0., log_c=0.) #write guesser for kernel parameters
kernel = kernel2
gp = celerite.GP(kernel, mean=model, fit_mean=True)
gp.compute(time, np.sqrt(counts))
initparams = gp.get_parameter_vector()
def loglikelihood(params):
    gp.set_parameter_vector(params)
    return gp.log_likelihood(counts)



def from_prior_m2(u):
    """
    Transform random samples from an n-dimensional uniform 
    distribution to the prior distribution
    """
    dt = 4.00 # time resolution of the light curve
    T = 4000.0 # total duration of the light curve

    log_a = 20 * u[0] - 10.0
    log_c = (np.log(T) + np.log(dt)) * u[1] + np.log(dt)
    
    #log_S0 = (np.log(1e7) + np.log(1))*u[0] + np.log(1.0)
    #log_Q = (np.log(20) + np.log(2.0)) * u[1] + np.log(2.0)
    #log_omega0 = (7-2)* u[2] - 7
    
    log_A = (np.log(1e7) + np.log(1))*u[2] - np.log(1.0)
    log_tau1 = (np.log(1e4) - np.log(1))*u[3] - np.log(1.0)
    log_tau2 = (np.log(T) - np.log(1/100. * T))*u[4] + np.log(1./100 * T)
    log_bkg = 20*u[4]-10.0
    
#    return np.array([log_a, log_c, log_S0, log_Q, log_omega0, log_A, log_tau1, log_tau2])
    return np.array([log_a, log_c, log_A, log_tau1, log_tau2, log_bkg])
    

ndim = len(gp.parameter_names)
print("number of dimensions: " + str(ndim))
sampler = dynesty.DynamicNestedSampler(loglikelihood, from_prior_m2, ndim, bound="multi", sample="rwalk", nlive=1000)

sampler.run_nested()



In [ ]:
res2 = sampler.results
lnz_truth = ndim * -np.log(2 * 10.)  # analytic evidence solution
fig, axes = dyplot.runplot(res2);  # summary (run) plot

fig, axes = dyplot.traceplot(res2, truths=trueparams,
                             truth_color='black', show_titles=True,
                             trace_cmap='viridis', connect=True,
                             connect_highlight=range(5))

samples, weights = res2.samples, np.exp(res2.logwt - res2.logz[-1])
mean, cov = dyfunc.mean_and_cov(samples, weights)

new_samples = dyfunc.resample_equal(samples, weights)
corner.corner(new_samples, truths=trueparams, labels=gp.get_parameter_names());
fig, ax = plt.subplots(1, 1, figsize=(10,5))

ax.errorbar(time, counts, yerr=counts_err, fmt="o", markersize=3, linewidth=1, c="black")
t = np.linspace(time[0], time[-1], 1000)


for s in new_samples[np.random.randint(len(new_samples), size=100)]:
    gp.set_parameter_vector(s)
    mu, var = gp.predict(counts, t, return_var=True)
    ax.plot(t, mu, lw=2, color=sns.color_palette()[1], alpha=0.2)
    
plt.tight_layout()

In [ ]:
bayes_fac1 = -5374.187
bayes_fac2 = -7263.976

In [ ]:
bayes_fac1 - bayes_fac2

Also seems to give quite strong evidence for the QPO model!

# Simulate from a OU process

Let's now do the reverse: simulate from a OU process, and compare the Bayes factors of those two models!

In [ ]:

log_a = 14.0 # QPP amplitude
log_c = -5 # QPP quality factor
log_omega0 = -3.0

# these should all be within priors!
realparams = [log_a, log_c] # OU process
#qpoparams = [log_s0, log_q0, log_omega0] # QPO
modelparams = [9.0, 6.92311406, 6.85207764, np.log(1000)] # flare model
mean_val = 1000.0
trueparams = np.hstack([realparams, modelparams, mean_val])#+ modelparams # + realparams # combined set of parameters
qpolabel = "A{:1.2f}".format(qpoparams[0])+"Q{:1.2f}".format(qpoparams[1])+"w{:1.2f}".format(qpoparams[2])

model = CTSModel_prior(log_A = modelparams[0], log_tau1 = modelparams[1], 
                       log_tau2 = modelparams[2], log_bkg = modelparams[3])
#kernel1 = qpp.SHOTerm_Prior(log_S0 = qpoparams[0], log_Q = qpoparams[1], log_omega0 = qpoparams[2])
kernel2 = qpp.RealTerm_Prior(log_a = realparams[0], log_c = realparams[1])
kernel = kernel2 #kernel2 + kernel1

dt = 4.0
tseg = 4000.0

time = np.linspace(0,tseg,tseg/dt)+0.01
#counts = qpp.simulate(time, model, kernel, noisy = True)

K = kernel.get_value(time[:, None] - time[None, :])
#y = np.abs(np.random.multivariate_normal(model.get_value(x), K))
y = np.abs(np.random.multivariate_normal(model.get_value(time), K))
yerr = 10
counts = np.random.normal(y, yerr)

counts_err = np.sqrt(counts)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9,4))
ax1.errorbar(time, counts, yerr=counts_err, fmt="o", markersize=8, lw=1, color="black")
ax1.plot(time, counts, lw=1, color="black", alpha=0.5)
ax1.set_xlim(time[0], time[-1])
ax1.set_ylim(0, np.max(counts)*1.1)
ax1.set_title("Full light curve")

ax2.errorbar(time, counts, yerr=counts_err, fmt="o", markersize=8, lw=1, color="black")
ax2.set_xlim(900, 1100)
ax2.set_title("short segment around peak")
ax2.set_ylim(np.max(counts)*0.9, np.max(counts)*1.1)

plt.tight_layout()

Now let's set up a QPO model:

In [ ]:

A_guess, t1_guess, t2_guess = qpp.initguess(time,counts)
model = CTSModel_prior(log_A = np.log(A_guess), log_tau1 = np.log(t1_guess), 
                           log_tau2 = np.log(t2_guess), log_bkg=np.log(1000))
kernel1 = qpp.SHOTerm_Prior(log_S0 =0.1, log_Q = 0.1, log_omega0 = 0.1) #write guesser for kernel parameters
#kernel2 = qpp.RealTerm_Prior(log_a=0., log_c=0.) #write guesser for kernel parameters
kernel = kernel1
gp = celerite.GP(kernel, mean=model, fit_mean=True)
gp.compute(time, np.sqrt(counts))
initparams = gp.get_parameter_vector()



def from_prior(u):
    """
    Transform random samples from an n-dimensional uniform 
    distribution to the prior distribution
    """
    dt = 4.00 # time resolution of the light curve
    T = 4000.0 # total duration of the light curve

    #log_a = 20 * u[0] - 10.0
    #log_c = (np.log(T) + np.log(dt)) * u[1] + np.log(dt)
    
    log_S0 = (np.log(1e7) + np.log(1))*u[0] + np.log(1.0)
    log_Q = (np.log(20) + np.log(2.0)) * u[1] + np.log(2.0)
    log_omega0 = (7-2)* u[2] - 7
    
    log_A = (np.log(1e7) + np.log(1))*u[3] - np.log(1.0)
    log_tau1 = (np.log(1e4) - np.log(1))*u[4] - np.log(1.0)
    log_tau2 = (np.log(T) - np.log(1/100. * T))*u[5] + np.log(1./100 * T)
    log_bkg = 20*u[6]-10.0
    
#    return np.array([log_a, log_c, log_S0, log_Q, log_omega0, log_A, log_tau1, log_tau2])
    return np.array([log_S0, log_Q, log_omega0, log_A, log_tau1, log_tau2, log_bkg])
    

def loglikelihood(params):
    gp.set_parameter_vector(params)
    return gp.log_likelihood(counts)




In [ ]:
ndim = len(gp.parameter_names)
print("number of dimensions: " + str(ndim))
sampler = dynesty.DynamicNestedSampler(loglikelihood, from_prior, ndim, bound="multi", sample="rwalk", nlive=1000)

sampler.run_nested()




In [ ]:
len(trueparams)

In [ ]:
res = sampler.results

lnz_truth = ndim * -np.log(2 * 10.)  # analytic evidence solution
fig, axes = dyplot.runplot(res);  # summary (run) plot



In [ ]:
res.samples.shape

In [ ]:
fig, axes = dyplot.traceplot(res, truths=trueparams,
                             truth_color='black', show_titles=True,
                             trace_cmap='viridis', connect=True,
                             connect_highlight=range(5))

In [ ]:
samples, weights = res.samples, np.exp(res.logwt - res.logz[-1])
mean, cov = dyfunc.mean_and_cov(samples, weights)


new_samples = dyfunc.resample_equal(samples, weights)
corner.corner(new_samples, truths=trueparams, labels=gp.get_parameter_names());

### Modelling with an OU Process

In [ ]:
A_guess, t1_guess, t2_guess = qpp.initguess(time,counts)
model = CTSModel_prior(log_A = np.log(A_guess), log_tau1 = np.log(t1_guess), 
                           log_tau2 = np.log(t2_guess), log_bkg=np.log(1000))
#kernel1 = qpp.SHOTerm_Prior(log_S0 =0.1, log_Q = 0.1, log_omega0 = 0.1) #write guesser for kernel parameters
kernel2 = qpp.RealTerm_Prior(log_a=0., log_c=0.) #write guesser for kernel parameters
kernel = kernel2
gp = celerite.GP(kernel, mean=model, fit_mean=True)
gp.compute(time, np.sqrt(counts))
initparams = gp.get_parameter_vector()
def loglikelihood(params):
    gp.set_parameter_vector(params)
    return gp.log_likelihood(counts)



def from_prior_m2(u):
    """
    Transform random samples from an n-dimensional uniform 
    distribution to the prior distribution
    """
    dt = 4.00 # time resolution of the light curve
    T = 4000.0 # total duration of the light curve

    log_a = 40 * u[0] - 20.0
    log_c = 14 * u[1] - 7.0
    
    #log_S0 = (np.log(1e7) + np.log(1))*u[0] + np.log(1.0)
    #log_Q = (np.log(20) + np.log(2.0)) * u[1] + np.log(2.0)
    #log_omega0 = (7-2)* u[2] - 7
    
    log_A = (np.log(1e7) + np.log(1))*u[2] - np.log(1.0)
    log_tau1 = (np.log(1e4) - np.log(1))*u[3] - np.log(1.0)
    log_tau2 = (np.log(T) - np.log(1/100. * T))*u[4] + np.log(1./100 * T)
    log_bkg = 20*u[4]-10.0
    
#    return np.array([log_a, log_c, log_S0, log_Q, log_omega0, log_A, log_tau1, log_tau2])
    return np.array([log_a, log_c, log_A, log_tau1, log_tau2, log_bkg])
    

ndim = len(gp.parameter_names)
print("number of dimensions: " + str(ndim))
sampler = dynesty.DynamicNestedSampler(loglikelihood, from_prior_m2, ndim, bound="multi", sample="rwalk", nlive=1000)

sampler.run_nested()



In [ ]:
res2 = sampler.results
lnz_truth = ndim * -np.log(2 * 10.)  # analytic evidence solution
fig, axes = dyplot.runplot(res2);  # summary (run) plot

fig, axes = dyplot.traceplot(res2, truths=trueparams,
                             truth_color='black', show_titles=True,
                             trace_cmap='viridis', connect=True,
                             connect_highlight=range(5))

samples, weights = res2.samples, np.exp(res2.logwt - res2.logz[-1])
mean, cov = dyfunc.mean_and_cov(samples, weights)

new_samples = dyfunc.resample_equal(samples, weights)
corner.corner(new_samples, truths=trueparams, labels=gp.get_parameter_names());
fig, ax = plt.subplots(1, 1, figsize=(10,5))

ax.errorbar(time, counts, yerr=counts_err, fmt="o", markersize=3, linewidth=1, c="black")
t = np.linspace(time[0], time[-1], 1000)


for s in new_samples[np.random.randint(len(new_samples), size=100)]:
    gp.set_parameter_vector(s)
    mu, var = gp.predict(counts, t, return_var=True)
    ax.plot(t, mu, lw=2, color=sns.color_palette()[1], alpha=0.2)
    
plt.tight_layout()

That looks pretty good. 

### A weaker OU Process Data set

Let's try a weaker OU process:

In [ ]:

log_a = 10.0 # QPP amplitude
log_c = -5 # QPP quality factor
log_omega0 = -3.0

# these should all be within priors!
realparams = [log_a, log_c] # OU process
#qpoparams = [log_s0, log_q0, log_omega0] # QPO
modelparams = [9.0, 6.92311406, 6.85207764, np.log(1000)] # flare model
mean_val = 1000.0
trueparams = np.hstack([realparams, modelparams, mean_val])#+ modelparams # + realparams # combined set of parameters
qpolabel = "A{:1.2f}".format(qpoparams[0])+"Q{:1.2f}".format(qpoparams[1])+"w{:1.2f}".format(qpoparams[2])

model = CTSModel_prior(log_A = modelparams[0], log_tau1 = modelparams[1], 
                       log_tau2 = modelparams[2], log_bkg = modelparams[3])
#kernel1 = qpp.SHOTerm_Prior(log_S0 = qpoparams[0], log_Q = qpoparams[1], log_omega0 = qpoparams[2])
kernel2 = qpp.RealTerm_Prior(log_a = realparams[0], log_c = realparams[1])
kernel = kernel2 #kernel2 + kernel1

dt = 4.0
tseg = 4000.0

time = np.linspace(0,tseg,tseg/dt)+0.01
#counts = qpp.simulate(time, model, kernel, noisy = True)

K = kernel.get_value(time[:, None] - time[None, :])
#y = np.abs(np.random.multivariate_normal(model.get_value(x), K))
y = np.abs(np.random.multivariate_normal(model.get_value(time), K))
yerr = 10
counts = np.random.normal(y, yerr)

counts_err = np.sqrt(counts)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9,4))
ax1.errorbar(time, counts, yerr=counts_err, fmt="o", markersize=8, lw=1, color="black")
ax1.plot(time, counts, lw=1, color="black", alpha=0.5)
ax1.set_xlim(time[0], time[-1])
ax1.set_ylim(0, np.max(counts)*1.1)
ax1.set_title("Full light curve")

ax2.errorbar(time, counts, yerr=counts_err, fmt="o", markersize=8, lw=1, color="black")
ax2.set_xlim(900, 1100)
ax2.set_title("short segment around peak")
ax2.set_ylim(np.max(counts)*0.9, np.max(counts)*1.1)

plt.tight_layout()

Again, we try the QPO model first:

In [ ]:

A_guess, t1_guess, t2_guess = qpp.initguess(time,counts)
model = CTSModel_prior(log_A = np.log(A_guess), log_tau1 = np.log(t1_guess), 
                           log_tau2 = np.log(t2_guess), log_bkg=np.log(1000))
kernel1 = qpp.SHOTerm_Prior(log_S0 =0.1, log_Q = 0.1, log_omega0 = 0.1) #write guesser for kernel parameters
#kernel2 = qpp.RealTerm_Prior(log_a=0., log_c=0.) #write guesser for kernel parameters
kernel = kernel1
gp = celerite.GP(kernel, mean=model, fit_mean=True)
gp.compute(time, np.sqrt(counts))
initparams = gp.get_parameter_vector()



def from_prior(u):
    """
    Transform random samples from an n-dimensional uniform 
    distribution to the prior distribution
    """
    dt = 4.00 # time resolution of the light curve
    T = 4000.0 # total duration of the light curve

    #log_a = 20 * u[0] - 10.0
    #log_c = (np.log(T) + np.log(dt)) * u[1] + np.log(dt)
    
    log_S0 = (np.log(1e7) + np.log(1))*u[0] + np.log(1.0)
    log_Q = (np.log(20) + np.log(2.0)) * u[1] + np.log(2.0)
    log_omega0 = (7-2)* u[2] - 7
    
    log_A = (np.log(1e7) + np.log(1))*u[3] - np.log(1.0)
    log_tau1 = (np.log(1e4) - np.log(1))*u[4] - np.log(1.0)
    log_tau2 = (np.log(T) - np.log(1/100. * T))*u[5] + np.log(1./100 * T)
    log_bkg = 20*u[6]-10.0
    
#    return np.array([log_a, log_c, log_S0, log_Q, log_omega0, log_A, log_tau1, log_tau2])
    return np.array([log_S0, log_Q, log_omega0, log_A, log_tau1, log_tau2, log_bkg])
    

def loglikelihood(params):
    gp.set_parameter_vector(params)
    return gp.log_likelihood(counts)

ndim = len(gp.parameter_names)
print("number of dimensions: " + str(ndim))
sampler = dynesty.DynamicNestedSampler(loglikelihood, from_prior, ndim, bound="multi", sample="rwalk", nlive=1000)

sampler.run_nested()


In [ ]:
res = sampler.results

lnz_truth = ndim * -np.log(2 * 10.)  # analytic evidence solution
fig, axes = dyplot.runplot(res);  # summary (run) plot




fig, axes = dyplot.traceplot(res, truths=trueparams,
                             truth_color='black', show_titles=True,
                             trace_cmap='viridis', connect=True,
                             connect_highlight=range(5))

samples, weights = res.samples, np.exp(res.logwt - res.logz[-1])
mean, cov = dyfunc.mean_and_cov(samples, weights)


new_samples = dyfunc.resample_equal(samples, weights)
corner.corner(new_samples, truths=trueparams, labels=gp.get_parameter_names());


fig, ax = plt.subplots(1, 1, figsize=(10,5))

ax.errorbar(time, counts, yerr=counts_err, fmt="o", markersize=3, linewidth=1, c="black")
t = np.linspace(time[0], time[-1], 1000)


for s in new_samples[np.random.randint(len(new_samples), size=100)]:
    gp.set_parameter_vector(s)
    mu, var = gp.predict(counts, t, return_var=True)
    ax.plot(t, mu, lw=2, color=sns.color_palette()[1], alpha=0.2)
    
plt.tight_layout()

### OU Model 

In [ ]:
A_guess, t1_guess, t2_guess = qpp.initguess(time,counts)
model = CTSModel_prior(log_A = np.log(A_guess), log_tau1 = np.log(t1_guess), 
                           log_tau2 = np.log(t2_guess), log_bkg=np.log(1000))
#kernel1 = qpp.SHOTerm_Prior(log_S0 =0.1, log_Q = 0.1, log_omega0 = 0.1) #write guesser for kernel parameters
kernel2 = qpp.RealTerm_Prior(log_a=0., log_c=0.) #write guesser for kernel parameters
kernel = kernel2
gp = celerite.GP(kernel, mean=model, fit_mean=True)
gp.compute(time, np.sqrt(counts))
initparams = gp.get_parameter_vector()
def loglikelihood(params):
    gp.set_parameter_vector(params)
    return gp.log_likelihood(counts)



def from_prior_m2(u):
    """
    Transform random samples from an n-dimensional uniform 
    distribution to the prior distribution
    """
    dt = 4.00 # time resolution of the light curve
    T = 4000.0 # total duration of the light curve

    log_a = 30 * u[0] - 15.0
    log_c = 14 * u[1] - 7.0
    
    #log_S0 = (np.log(1e7) + np.log(1))*u[0] + np.log(1.0)
    #log_Q = (np.log(20) + np.log(2.0)) * u[1] + np.log(2.0)
    #log_omega0 = (7-2)* u[2] - 7
    
    log_A = (np.log(1e7) + np.log(1))*u[2] - np.log(1.0)
    log_tau1 = (np.log(1e4) - np.log(1))*u[3] - np.log(1.0)
    log_tau2 = (np.log(T) - np.log(1/100. * T))*u[4] + np.log(1./100 * T)
    log_bkg = 20*u[4]-10.0
    
#    return np.array([log_a, log_c, log_S0, log_Q, log_omega0, log_A, log_tau1, log_tau2])
    return np.array([log_a, log_c, log_A, log_tau1, log_tau2, log_bkg])
    

ndim = len(gp.parameter_names)
print("number of dimensions: " + str(ndim))
sampler = dynesty.DynamicNestedSampler(loglikelihood, from_prior_m2, ndim, bound="multi", sample="rwalk", nlive=2000)

sampler.run_nested()




In [ ]:
res2 = sampler.results
lnz_truth = ndim * -np.log(2 * 10.)  # analytic evidence solution
fig, axes = dyplot.runplot(res2);  # summary (run) plot

fig, axes = dyplot.traceplot(res2, truths=trueparams,
                             truth_color='black', show_titles=True,
                             trace_cmap='viridis', connect=True,
                             connect_highlight=range(5))

samples, weights = res2.samples, np.exp(res2.logwt - res2.logz[-1])
mean, cov = dyfunc.mean_and_cov(samples, weights)

new_samples = dyfunc.resample_equal(samples, weights)
corner.corner(new_samples, truths=trueparams, labels=gp.get_parameter_names());
fig, ax = plt.subplots(1, 1, figsize=(10,5))

ax.errorbar(time, counts, yerr=counts_err, fmt="o", markersize=3, linewidth=1, c="black")
t = np.linspace(time[0], time[-1], 1000)


for s in new_samples[np.random.randint(len(new_samples), size=100)]:
    gp.set_parameter_vector(s)
    mu, var = gp.predict(counts, t, return_var=True)
    ax.plot(t, mu, lw=2, color=sns.color_palette()[1], alpha=0.2)
    
plt.tight_layout()